# Comparing transcripts across a single company
Using Spacy and a regex for numerics

In [1]:
import spacy
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
from aux import extract_call_transcript

# Load core Roberta 
model = spacy.load("en_core_web_trf")
quarter_1 = extract_call_transcript('NVDA', 1, 2021)
quarter_2 = extract_call_transcript('NVDA', 2, 2021)

In [2]:
from aux import nlp_tag_sentences_by_characters
doc_quarter_1, quarter_1_matches = nlp_tag_sentences_by_characters(quarter_1, model)
doc_quarter_2, quarter_2_matches = nlp_tag_sentences_by_characters(quarter_2, model)

In [18]:
text_q = ""
for i, quarter in enumerate([quarter_1_matches, quarter_2_matches]):
    print(f'{i+1}/{2}')
    for sent in quarter:
        #print(sent[1])
        print(len(sent[1]))
        text_q +=sent[1]

1/2
165
165
75
142
239
71
83
35
139
138
163
139
104
46
175
113
80
110
117
132
118
99
83
55
48
73
116
214
160
169
129
84
73
71
52
54
69
55
2/2
98
137
247
122
92
157
74
109
209
84
99
65
143
50
49
47
55
136
110
115
116
84
99
83
84
84


In [19]:
text_q_1 = ' '.join(sent[1] for sent in quarter_1_matches)
text_q_2 = ' '.join(sent[1] for sent in quarter_2_matches)

In [23]:
text_q = text_q_1 + " " + text_q_2


In [28]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch

#tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
#model = AutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")



questions = [
    "How does GAAP margin compares between quarter 1 and quarter 2 for NVIDIA?",
    "Did the company beat revenue expectations?",
    "How were the gross margins for each of the quarters?",
]

for question in questions:
    inputs = tokenizer(question, text_q, add_special_tokens=True, return_tensors="pt", truncation=True)
    input_ids = inputs["input_ids"].tolist()[0]
    outputs = model(**inputs)
    answer_start_scores = outputs.start_logits
    answer_end_scores = outputs.end_logits
    # Get the most likely beginning of answer with the argmax of the score
    answer_start = torch.argmax(answer_start_scores)
    # Get the most likely end of answer with the argmax of the score
    answer_end = torch.argmax(answer_end_scores) + 1
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))
    print(f"Question: {question}")
    print(f"Answer: {answer}")

Question: How does GAAP margin compares between quarter 1 and quarter 2 for NVIDIA?
Answer: mid - to high 60 % range
Question: Did the company beat revenue expectations?
Answer: slightly ahead of our outlook
Question: How were the gross margins for each of the quarters?
Answer: 65. 1 % and non - gaap was 65. 8 %
